In [1]:
!nvidia-smi

Sun Feb  8 06:46:33 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [7]:
!pip uninstall -y torch torchvision torchaudio mamba-ssm causal-conv1d

!pip install torch==2.7.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

!pip install https://github.com/Dao-AILab/causal-conv1d/releases/download/v1.6.0/causal_conv1d-1.6.0+cu12torch2.7cxx11abiFALSE-cp312-cp312-linux_x86_64.whl

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Found existing installation: mamba-ssm 2.3.0
Uninstalling mamba-ssm-2.3.0:
  Successfully uninstalled mamba-ssm-2.3.0
Found existing installation: causal-conv1d 1.6.0
Uninstalling causal-conv1d-1.6.0:
  Successfully uninstalled causal-conv1d-1.6.0
Looking in indexes: https://download.pytorch.org/whl/cu126
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 125.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 238.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 206.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 MB 5.2 MB/s eta 0:00:0000:0100:01


In [1]:
!pip install https://github.com/state-spaces/mamba/releases/download/v2.3.0/mamba_ssm-2.3.0+cu12torch2.7cxx11abiFALSE-cp312-cp312-linux_x86_64.whl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 533.3/533.3 MB 2.4 MB/s eta 0:00:0000:0100:01


In [19]:
!git clone https://github.com/Taya-san/SKRIPSIAHMOMENT.git

Cloning into 'SKRIPSIAHMOMENT'...
remote: Enumerating objects: 103, done.
remote: Counting objects: 100% (103/103), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 103 (delta 53), reused 76 (delta 26), pack-reused 0 (from 0)
Receiving objects: 100% (103/103), 799.79 KiB | 3.46 MiB/s, done.
Resolving deltas: 100% (53/53), done.


In [69]:
%cd SKRIPSIAHMOMENT/
%ls
!git pull

[Errno 2] No such file or directory: 'SKRIPSIAHMOMENT/'
/content/SKRIPSIAHMOMENT
mamba_quin/  notebooks+pdf/  README.md  utils/
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 320 bytes | 320.00 KiB/s, done.
From https://github.com/Taya-san/SKRIPSIAHMOMENT
   d20b81f..90fbe42  main       -> origin/main
Updating d20b81f..90fbe42
Fast-forward
 utils/trainer.py | 1 -
 1 file changed, 1 deletion(-)


In [21]:
import torch
from torch import nn
import numpy as np
import transformers
from transformers import AutoModel, AutoConfig, AutoTokenizer
from datasets import load_dataset
import sklearn.model_selection
import mamba_ssm
try:
    from mamba_ssm.ops.triton.selective_state_update import selective_state_update
    from mamba_ssm.ops.selective_scan_interface import selective_scan_fn, mamba_inner_fn
    
    mamba_ssm.selective_state_update = selective_state_update
    mamba_ssm.selective_scan_fn = selective_scan_fn
    mamba_ssm.mamba_inner_fn = mamba_inner_fn

except ImportError as e:
    print(f"Patch failed! Error: {e}")

In [70]:
from mamba_quin import MambaQuin, MambaQuinConfig
from utils import train_modelnoclt

In [52]:
config = MambaQuinConfig(
    backbone_model = "state-spaces/mamba-130m",
    num_classes = 2
)

model = AutoModel.from_config(config)

Loading weights:   0%|          | 0/241 [00:00<?, ?it/s]

MambaModel LOAD REPORT from: state-spaces/mamba-130m
Key                                                           | Status     | 
--------------------------------------------------------------+------------+-
backbone.embedding.weight                                     | UNEXPECTED | 
lm_head.weight                                                | UNEXPECTED | 
layers.{24, 25, 26, 27, 28, 29, 30, 31}.mixer.in_proj.weight  | MISSING    | 
layers.{24, 25, 26, 27, 28, 29, 30, 31}.mixer.D               | MISSING    | 
layers.{24, 25, 26, 27, 28, 29, 30, 31}.mixer.out_proj.weight | MISSING    | 
layers.{24, 25, 26, 27, 28, 29, 30, 31}.mixer.dt_proj.bias    | MISSING    | 
layers.{24, 25, 26, 27, 28, 29, 30, 31}.norm.weight           | MISSING    | 
layers.{24, 25, 26, 27, 28, 29, 30, 31}.mixer.x_proj.weight   | MISSING    | 
layers.{24, 25, 26, 27, 28, 29, 30, 31}.mixer.dt_proj.weight  | MISSING    | 
layers.{24, 25, 26, 27, 28, 29, 30, 31}.mixer.A_log           | MISSING    | 
embeddings.

In [53]:
dataset = load_dataset("stanfordnlp/imdb")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [56]:
tokenizer = AutoTokenizer.from_pretrained(config.backbone_model)

In [71]:
train_modelnoclt(model, tokenizer, 3, dataset_dict=dataset, batch_size=64, tr_data=None, ts_data=None)

Epoch 1/3:   0%|          | 0/391 [00:00<?, ?it/s]

Epoch 1/3:   0%|          | 0/391 [00:00<?, ?it/s]


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [75]:
from datasets import Dataset
from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader
from tqdm import tqdm
import warnings
import torch.optim as optim
device = 'cuda'
batch_size = 64
optimizer_type = 'adam'
if dataset is not None :
    training_data = dataset['train']
    test_data = dataset['test']
else:
    training_data = Dataset.from_dict(tr_data)
    test_data = Dataset.from_dict(ts_data)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def tokenizer_fn(batch):
    return tokenizer(
        batch["text"],
        truncation = True,
        max_length = 256,
        padding = False
    )

training_data = training_data.map(
        tokenizer_fn,
        batched = True,
)
test_data = test_data.map(
        tokenizer_fn,
        batched = True,
)


training_data = training_data.rename_column('label', 'labels')
test_data = test_data.rename_column('label', 'labels')

training_data = training_data.remove_columns(['text'])
test_data = test_data.remove_columns(['text'])

training_data.set_format(
        type = 'torch',
        columns = ['input_ids', 'attention_mask', 'labels']
)
test_data.set_format(
        type = 'torch',
        columns = ['input_ids', 'attention_mask', 'labels']
)

collator = DataCollatorWithPadding(tokenizer)

training_loader = DataLoader(
        training_data,
        batch_size = batch_size,
        shuffle = True,
        pin_memory = True,
        collate_fn = collator
)
test_loader = DataLoader(
        test_data,
        batch_size = batch_size,
        shuffle = False,
        pin_memory = True,
        collate_fn = collator
)


if optimizer_type == 'adam':
    optimizer = optim.Adam(model.parameters(), lr = 1e-3)
elif optimizer_type == 'sgd':
    optimizer = optim.SGD(model.parameters(), lr = 1e-3)

model.float()
model.to(device)

model.config.epochs_loss_log = getattr(model.config, "epochs_loss_log", [])

for epoch in range(1):
    model.train()
    running_loss = 0.0

    loop = tqdm(training_loader, desc=f'Epoch {epoch + 1}/{1}', leave=True)

    for batch in loop:

        batch = {k:v.to(device) for k,v in batch.items()}
        print(batch)
#         outputs = model(**batch)

#         optimizer.zero_grad()

#         loss = outputs.loss
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()
#         loop.set_postfix(loss=loss.item())

#     epoch_loss = running_loss/len(training_loader)
#     model.config.epochs_loss_log.append(epoch_loss)
#     print(f'Epoch {epoch + 1} loss: {epoch_loss:.4f}')




Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Epoch 1/1:  15%|█▍        | 57/391 [00:00<00:01, 283.31it/s]

{'labels': tensor([1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
        0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0,
        0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0], device='cuda:0'), 'input_ids': tensor([], device='cuda:0', size=(64, 0), dtype=torch.int64), 'attention_mask': tensor([], device='cuda:0', size=(64, 0), dtype=torch.int64)}
{'labels': tensor([0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1,
        1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,
        1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1], device='cuda:0'), 'input_ids': tensor([], device='cuda:0', size=(64, 0), dtype=torch.int64), 'attention_mask': tensor([], device='cuda:0', size=(64, 0), dtype=torch.int64)}
{'labels': tensor([0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0,
        1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
        1, 1, 0, 0, 1, 0, 0

Epoch 1/1:  22%|██▏       | 86/391 [00:00<00:01, 277.72it/s]

...

{'labels': tensor([0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1,
        1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
        0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1], device='cuda:0'), 'input_ids': tensor([], device='cuda:0', size=(64, 0), dtype=torch.int64), 'attention_mask': tensor([], device='cuda:0', size=(64, 0), dtype=torch.int64)}
{'labels': tensor([0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0,
        1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0], device='cuda:0'), 'input_ids': tensor([], device='cuda:0', size=(64, 0), dtype=torch.int64), 'attention_mask': tensor([], device='cuda:0', size=(64, 0), dtype=torch.int64)}
{'labels': tensor([0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0,
        0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1,
        0, 1, 0, 1, 0, 0, 0

Epoch 1/1:  82%|████████▏ | 320/391 [00:01<00:00, 283.45it/s]

{'labels': tensor([0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1,
        1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0,
        1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0], device='cuda:0'), 'input_ids': tensor([], device='cuda:0', size=(64, 0), dtype=torch.int64), 'attention_mask': tensor([], device='cuda:0', size=(64, 0), dtype=torch.int64)}
{'labels': tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0,
        1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0,
        0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1], device='cuda:0'), 'input_ids': tensor([], device='cuda:0', size=(64, 0), dtype=torch.int64), 'attention_mask': tensor([], device='cuda:0', size=(64, 0), dtype=torch.int64)}
{'labels': tensor([1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0,
        1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
        1, 0, 1, 0, 1, 1, 0

Epoch 1/1: 100%|██████████| 391/391 [00:01<00:00, 279.55it/s]

{'labels': tensor([0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1,
        0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1,
        1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0], device='cuda:0'), 'input_ids': tensor([], device='cuda:0', size=(64, 0), dtype=torch.int64), 'attention_mask': tensor([], device='cuda:0', size=(64, 0), dtype=torch.int64)}
{'labels': tensor([1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
        0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1,
        0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1], device='cuda:0'), 'input_ids': tensor([], device='cuda:0', size=(64, 0), dtype=torch.int64), 'attention_mask': tensor([], device='cuda:0', size=(64, 0), dtype=torch.int64)}
{'labels': tensor([0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0,
        1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0,
        1, 1, 1, 1, 0, 1, 1